# 04 • Construção do SQLite

Objetivo: criar `data/processed/imoveis_rurais.sqlite` com índices para o Streamlit.

Entrada: `imoveis_er_tratado.csv`


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re, unicodedata

# Caminhos do projeto (ajuste se necessário)
BASE_DIR = Path.cwd().resolve().parents[0]  # .../notebooks -> projeto
RAW_DIR = BASE_DIR / "data" / "raw"
PROCESSED_DIR = BASE_DIR / "data" / "processed"
OUTPUTS_DIR = BASE_DIR / "outputs"

RAW_DIR, PROCESSED_DIR, OUTPUTS_DIR


(WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/raw'),
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/processed'),
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/outputs'))

In [2]:
import sqlite3

In [3]:
in_csv = PROCESSED_DIR / "imoveis_er_tratado.csv"
db_path = PROCESSED_DIR / "imoveis_rurais.sqlite"
in_csv.exists(), db_path

(True,
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/processed/imoveis_rurais.sqlite'))

In [9]:
# Recriar banco
if db_path.exists():
    db_path.unlink()

conn = sqlite3.connect(db_path)
cur = conn.cursor()
cur.execute(
    "CREATE TABLE imoveis ("
    "codigo_imovel TEXT, denominacao TEXT, denominacao_norm TEXT, "
    "ibge_municipio INTEGER, municipio TEXT, municipio_norm TEXT, uf TEXT, "
    "area_total_ha REAL, titular TEXT, titular_norm TEXT, "
    "natureza_juridica TEXT, condicao_pessoa TEXT, percentual_detencao REAL, pais TEXT"
    ");"
)
conn.commit()

In [10]:
df = pd.read_csv(in_csv, sep=";", dtype=str, low_memory=False)

to_db = pd.DataFrame({
    "codigo_imovel": df["CÓDIGO DO IMOVEL"],
    "denominacao": df["DENOMIÇÃO DO IMÓVEL"],
    "denominacao_norm": df["DENOMINACAO_NORM"],
    "ibge_municipio": pd.to_numeric(df["IBGE_MUNICIPIO"], errors="coerce"),
    "municipio": df["MUNICÍPIO"],
    "municipio_norm": df["MUNICIPIO_NORM"],
    "uf": df["UF"],
    "area_total_ha": pd.to_numeric(df["AREA_TOTAL_HA"], errors="coerce"),
    "titular": df["TITULAR"],
    "titular_norm": df["TITULAR_NORM"],
    "natureza_juridica": df["NATUREZA JURÍDICA"],
    "condicao_pessoa": df["CONDIÇÃO DA PESSOA"],
    "percentual_detencao": pd.to_numeric(df["PERCENTUAL_DETENCAO"], errors="coerce"),
    "pais": df["PAÍS"],
})

to_db.to_sql("imoveis", conn, if_exists="append", index=False)
conn.commit()
len(to_db)

26332

In [11]:
cur.execute("CREATE INDEX idx_municipio_norm ON imoveis (municipio_norm);")
cur.execute("CREATE INDEX idx_area ON imoveis (area_total_ha);")
cur.execute("CREATE INDEX idx_titular_norm ON imoveis (titular_norm);")
cur.execute("CREATE INDEX idx_denominacao_norm ON imoveis (denominacao_norm);")
conn.commit()
conn.close()

db_path

WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/processed/imoveis_rurais.sqlite')

In [12]:
conn = sqlite3.connect(db_path)
df_q = pd.read_sql_query(
    "SELECT municipio, COUNT(*) as n, ROUND(AVG(area_total_ha),2) as area_media "
    "FROM imoveis GROUP BY municipio ORDER BY n DESC LIMIT 20;",
    conn
)
conn.close()
df_q

,municipio,n,area_media
0,BRAGANÇA PAULISTA,5194,16.58
1,JUNDIAÍ,4500,11.90
2,ITATIBA,2577,18.51
3,JARINU,2367,13.21
4,PINHALZINHO,1914,11.51
5,PEDRA BELA,1500,16.08
6,ITUPEVA,1317,23.12
7,CABREÚVA,1175,40.17
8,LOUVEIRA,1011,12.77
9,MORUNGABA,987,20.78
